<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

This notebook identifies the keywords to use for picking out crimes corresponding to a number of mandatory minimums in VA. It uses court data publicly available at https://virginiacourtdata.org/.

In [1]:
import os
import sys
sys.path.insert(0, os.path.relpath('../../')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from super_simulation.super_simulation_factory import SuperSimulationFactory
from transition_table import TransitionTable
from shell_compartment import ShellCompartment
from spark_policy import SparkPolicy
from utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [2]:
court_data = pd.concat([pd.read_csv(f'../../state/VA/VA_data/VA_jail_data/circuit_criminal_{year}_anon_00.csv') for year in range(2010,2020)])
                      
court_data = court_data.drop(
    ['HearingJury', 'HearingPlea', 'HearingRoom', 'fips', 'Locality', 'DrivingRestrictions', 
     'RestrictionEffectiveDate', 'RestrictionEndDate', 'RestitutionPaid', 
     'RestitutionAmount', 'Military', 'TrafficFatality', 'Sex', 'Race', 'Address', 'Costs', 
     'FinesCostPaid', 'ProgramType', 'CourtDMVSurrender', 'DriverImprovementClinic', 'VAAlcoholSafetyAction', 
     'OffenseDate', 'LifeDeath', 'OperatorLicenseSuspensionTime', 'FineAmount', 'ArrestDate',
     'AmendedCharge', 'AmendedCodeSection', 'AmendedChargeType', 'ConcludedBy', 'DispositionDate'], axis=1)
court_data = court_data[court_data.DispositionCode == 'Guilty'].drop('DispositionCode', axis=1)
print(f"number of records: {len(court_data)}")
court_data.head()

court_data = court_data[(court_data.HearingResult == 'Sent') | (court_data.HearingResult == 'Sentenced')]
court_data.SentenceTime = court_data.SentenceTime.fillna(0)
court_data.SentenceSuspended = court_data.SentenceSuspended.fillna(0)
court_data['effective_sentence_months'] = (court_data.SentenceTime - court_data.SentenceSuspended) / 365 * 12
court_data = court_data[court_data.Charge.notnull()]




/var/folders/3l/phmqvp1s6y5_l56qp_wwfrb80000gn/T/ipykernel_18383/3681986318.py:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  court_data = pd.concat([pd.read_csv(f'../../state/VA/VA_data/VA_jail_data/circuit_criminal_{year}_anon_00.csv') for year in range(2010,2020)])
/var/folders/3l/phmqvp1s6y5_l56qp_wwfrb80000gn/T/ipykernel_18383/3681986318.py:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  court_data = pd.concat([pd.read_csv(f'../../state/VA/VA_data/VA_jail_data/circuit_criminal_{year}_anon_00.csv') for year in range(2010,2020)])
/var/folders/3l/phmqvp1s6y5_l56qp_wwfrb80000gn/T/ipykernel_18383/3681986318.py:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  court_data = pd.concat([pd.read_csv(f'../../state/VA/VA_data/VA_jail_data/circuit_criminal_{year}_anon_00.csv') for year in range(2010,2020)])
/var/folders/3l/

number of records: 791960


In [3]:
court_data['day'] = court_data.HearingDate.apply(lambda x: int(x.split('-')[2]))
court_data['month'] = court_data.HearingDate.apply(lambda x: int(x.split('-')[1]))
court_data['year'] = court_data.HearingDate.apply(lambda x: int(x.split('-')[0]))

In [ ]:
def get_most_serious_case(df):
    most_serious_crime = df.sort_values('effective_sentence_months').iloc[-1]
    most_serious_crime['crimes'] = len(df)
#     most_serious_crime.DRUG_POSSESSION = df.DRUG_POSSESSION.any()
#     most_serious_crime.DRUG_SALE = df.DRUG_SALE.any()
#     most_serious_crime.DRUG_OTHER = df.DRUG_OTHER.any()
#     most_serious_crime.MARIJUANA = df.MARIJUANA.any()
    return most_serious_crime

court_data = court_data.groupby(['person_id', 'HearingDate'], as_index=False).apply(get_most_serious_case)

In [3]:
mms_stats = pd.read_csv('../../state/VA/2022/CurrentMMs.csv')
mms_stats = mms_stats[mms_stats.VCC.notnull()]
mms_stats['offense_code'] = mms_stats.VCC.apply(lambda x: x.split('-')[0] + x.split('-')[1])
mms_stats

,CHAPTER,CHAPDEC,SECTION,SCPUNCT,SUBSECT,PARAGRF,STATUTE,EFFDATE,DateAdd,SGOFF,SENTENCE,MAN_MIN,MAN_UNIT,RETIREDATE,MM,MM_clean,VCC,IBRCODE,FelonyTimeServed,HEADING,SUBHEAD,DESCRTN,offense_code
0,3,2,4212,NaN,NaN,"D,i","3.2-4212(D,i)",7/1/09,7/1/09,N,0-12M,90,D,NaN,90 D,3,AGR-4632-M1,NaN,0,"AGRICULTURE, HORTICULTURE & FOOD",Tobacco Product Manufacturers,Sell or distribute cigarettes not in directory...,AGR4632
1,3,2,4212,NaN,NaN,"D,ii","3.2-4212(D,ii)",7/1/09,7/1/09,N,0-12M,90,D,NaN,90 D,3,AGR-4634-M1,NaN,0,"AGRICULTURE, HORTICULTURE & FOOD",Tobacco Product Manufacturers,"Possess, import, etc., cigarettes not in direc...",AGR4634
2,4,1,302,.,NaN,NaN,4.1-302,NaN,4/28/94,N,0-12M,30,D,NaN,30 D,1,ALC-4163-M1,NaN,0,ALCOHOL,Sell,"Sale, illegal alcohol - subsequent",ALC4163
3,16,1,253,.,2,A,16.1-253.2(A),7/1/16,7/1/16,N,0-12M,60,D,NaN,60 D,2,PRT-5001-M1,NaN,0,PROTECTIVE ORDERS,Juvenile & Domestic Court,Violation of protective order (violence) 2nd w...,PRT5001
4,16,1,253,.,2,A,16.1-253.2(A),7/1/16,7/1/16,N,1Y-5Y (II),6,M,NaN,6 M,6,PRT-5002-F6,NaN,1,PROTECTIVE ORDERS,Juvenile & Domestic Court,Violation of protective order (violence) 3rd w...,PRT5002
5,18,2,31,NaN,NaN,6,18.2-31(6),7/1/21,7/1/21,N,Life-Life (I),NaN,Life,NaN,Life,600,MUR-0923-F1,NaN,1,MURDER,Aggravated,Law enforcement officer,MUR0923
6,18,2,36,.,1,B,18.2-36.1(B),NaN,NaN,Y,1Y-20Y (II),1,Y,NaN,1 Y,12,MUR-0948-F9,NaN,3,MURDER,Manslaughter,"Involuntary manslaughter, under the influence ...",MUR0948
7,18,2,36,.,2,B,18.2-36.2(B),7/1/05,7/2/05,N,1Y-20Y (II),1,Y,NaN,1 Y,12,MUR-0949-F9,NaN,3,MURDER,Manslaughter,"Involuntary manslaughter, under the influence ...",MUR0949
8,18,2,46,.,2,NaN,18.2-46.2,7/1/05,7/1/05,N,1Y-10Y,2,Y,NaN,2 Y,24,MOB-1280-F5,NaN,3,GANGS,"Participation - School Zone, Community Center,...",Participation in crime for benefit/direction o...,MOB1280
9,18,2,46,.,2,NaN,18.2-46.2,7/1/05,7/1/05,N,2Y-10Y,2,Y,NaN,2 Y,24,MOB-1281-F4,NaN,3,GANGS,"Participation - School Zone, Community Center,...",Participation in crime for gang that includes ...,MOB1281


In [4]:
raw_va_sentence_df = pd.read_csv(
    '../../state/VA/VA_data/unprocessed_va_historical_sentences_v2.csv',
    sep='\t'
)
raw_va_sentence_df['crime_type'] = raw_va_sentence_df['Offense Group'].ffill()
raw_va_sentence_df['offense_code'] = raw_va_sentence_df['VCC'].ffill()
raw_va_sentence_df['crime'] = raw_va_sentence_df['Off1VCC'].ffill()
raw_va_sentence_df['judge_id'] = raw_va_sentence_df['JudgeID'].ffill()
raw_va_sentence_df['sentence_type_code'] = raw_va_sentence_df['ActDisp'].ffill()
raw_va_sentence_df['effective_sentence_months'] = raw_va_sentence_df['effsent']
raw_va_sentence_df['fiscal_year'] = raw_va_sentence_df['FiscalYr'].ffill()
raw_va_sentence_df['life_sentence'] = raw_va_sentence_df['EffLif']
raw_va_sentence_df = raw_va_sentence_df.rename({'Off1Date':'offense_date'}, axis=1)

raw_va_sentence_df = raw_va_sentence_df[~raw_va_sentence_df['crime_type'].str.contains('Total')]


raw_va_sentence_df = raw_va_sentence_df.drop(
    ['Offense Group', 'VCC', 'Off1VCC', 'JudgeID', 'ActDisp', 'effsent', 'FiscalYr', 'EffLif'],
    axis=1
)
raw_va_sentence_df

,offense_date,OffLName,OffFName,crime_type,offense_code,crime,judge_id,sentence_type_code,effective_sentence_months,fiscal_year,life_sentence
0,5/20/2017,MOORE,KENNETH,ASSAULT,ASL1316,ASL1316F6,0,3.0,18.0,2018.0,0.0
1,4/25/2018,CLINE,MATTHEW,ASSAULT,ASL1316,ASL1316F6,0,3.0,10.0,2019.0,0.0
2,11/11/2017,TINSLEY,JAVON,ASSAULT,ASL1316,ASL1316F6,10DB,1.0,0.0,2018.0,0.0
3,6/8/2015,COMPTON,ROBERT,ASSAULT,ASL1316,ASL1316F6,10DB,2.0,0.0,2017.0,0.0
4,12/25/2016,MILLER,MARCUS,ASSAULT,ASL1316,ASL1316F6,10DB,3.0,7.0,2017.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
74313,6/1/2017,CUNNINGHAM,JACOB,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9MM,3.0,8.0,2018.0,0.0
74314,6/27/2017,STURDIVANT,JUSTIN,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9MM,3.0,13.0,2018.0,0.0
74315,11/30/2017,MAINES,BRADY,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9MM,3.0,7.0,2019.0,0.0
74316,12/26/2016,MORRIS,DAVID,WEAPONS/FIREARMS,WPN5298,WPN5298F6,9TH,2.0,5.0,2018.0,0.0


In [ ]:
def count_instances(crime_code):
    return len(raw_va_sentence_df[raw_va_sentence_df.offense_code == crime_code])

mms_stats['instances'] = mms_stats.offense_code.apply(count_instances)
# plt.hist(mms_stats.instances / mms_stats.instances.sum(), bins=40)
# plt.yscale('log')
mms_stats.sort_values('instances', ascending=False)

In [ ]:
court_data['ASSAULT_ON_POLICE'] = court_data.Charge.apply(
    lambda x: any([i in x for i in ['LAW ENF', 'POLICE']]) and not any([i in x for i in [
        'ELUDE', 'FALSE', 'ELUDING'
    ]])
)

court_data['THIRD_DWI_10YR'] = court_data.Charge.apply(
    lambda x: any([i in x for i in ['3RD', '3+', 'THREE', 'THIRD']]) and not any([i in x for i in [
        'SHOPLIFT', 'A&B', 'LARC', 'ASSAULT', 'SHOPLFT', 'SCH I', 'SCH I', 'CONCEALMENT', 'PROTECTIVE', 
        'POSS', 'INDECENT EXPOSURE', 'DIST'
    ]])
)

court_data['DRUG_INTENT'] = court_data.Charge.apply(
    lambda x: any([i in x for i in [
        'INTENT',
        'W/INT',
        'PWI']]) 
    and not any([i in x for i in [
        'PORN', 'GUN', 'WEAPON', 'FIREARM', 'CELL PHONE', 'ALCOHOL', 'CIG']])
)
court_data['DRUG_SALE'] = court_data.Charge.apply(
    lambda x: any([i in x for i in [
        'DISTRB',
        'DISTRIBUTION',
        'DISTRIB',
        'DIST',
        'PWID', 
        'SALE',
        'SELL']])
        and not any([i in x for i in [
        'PORN', 'GUN', 'WEAPON', 'FIREARM', 'CELL PHONE', 'ALCOHOL', 'CIG']])
)
court_data['2ND'] = court_data.Charge.apply(
    lambda x: any([i in x for i in ['2ND', 'SECOND']])
)
court_data['SECOND_PWID'] = (court_data['DRUG_INTENT'] | court_data['DRUG_SALE']) & court_data['2ND']

court_data = court_data.drop(
    ['DRUG_INTENT', 'DRUG_SALE', '2ND'], axis=1)
# court_data['DRUG'] = court_data.Charge.apply(
#     lambda x: any([i in x for i in [
#         'POSS',
#         'CONTROLLED SUBSTANCE',
#         'SCH 1',
#         'DISTRB',
#         'DISTRIB',
#         'DISTRIBUTION',
#         'SELL'
#         'PWID', 
#         'MARIJUANA',
#         'MARIJ',
#         'COCAINE',
#         'HEROIN',
#         'DRUG'
#     ]]) and not any([i in x for i in [
#         'PORN', 'GUN', 'WEAPON', 'FIREARM', 'CELL PHONE', 'ALCOHOL'
#     ]])
# )

# court_data.DRUG_POSSESSION = court_data.DRUG_POSSESSION & court_data.DRUG


# court_data.DRUG_SALE = court_data.DRUG_SALE & court_data.DRUG

# court_data['DRUG_OTHER'] = court_data.DRUG & ~(court_data.DRUG_SALE | court_data.DRUG_POSSESSION)



court_data[court_data.SECOND_PWID].groupby('Charge').count().sort_values('HearingType', ascending=False).iloc[:200]

In [ ]:
nar_data = court_data[court_data.SECOND_PWID]
nar_data[(nar_data.year == 2017) | (nar_data.year == 2018) | (nar_data.year == 2019)]
nar_data